In [37]:
import os
import sys

import logging
import re
import pathlib

import datetime

import numpy as np
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt

from pyplink import PyPlink
import seaborn as sns

import statsmodels.api as sm

import matplotlib.patches as patches
import matplotlib

from basic_tools import *

"""

jupyter nbconvert 5_association.ipynb --to script
python 5_association.py albumin 2 0

for i in {00..101};do python 5_association.py $i 2 1;done


for i in {00..10};do python 5_association.py $i 2 0;done
for i in {11..20};do python 5_association.py $i 2 0;done
for i in {21..30};do python 5_association.py $i 2 0;done
for i in {31..40};do python 5_association.py $i 2 0;done
for i in {41..50};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {61..70};do python 5_association.py $i 2 0;done
for i in {71..80};do python 5_association.py $i 2 0;done
for i in {91..90};do python 5_association.py $i 2 0;done
for i in {91..101};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done

for i in {00..101};do python 5_association.py $i 1 0,2;done
for i in {00..101};do python 5_association.py $i 1 1;done

for i in {00..20};do python 5_association.py $i 1 1;done
for i in {21..40};do python 5_association.py $i 1 1;done
for i in {41..60};do python 5_association.py $i 1 1;done
for i in {61..80};do python 5_association.py $i 1 1;done
for i in {81..101};do python 5_association.py $i 1 1;done

"""

'\n\njupyter nbconvert 5_association.ipynb --to script\npython 5_association.py albumin 2 0\n\nfor i in {00..101};do python 5_association.py $i 2 1;done\n\n\nfor i in {00..10};do python 5_association.py $i 2 0;done\nfor i in {11..20};do python 5_association.py $i 2 0;done\nfor i in {21..30};do python 5_association.py $i 2 0;done\nfor i in {31..40};do python 5_association.py $i 2 0;done\nfor i in {41..50};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {61..70};do python 5_association.py $i 2 0;done\nfor i in {71..80};do python 5_association.py $i 2 0;done\nfor i in {91..90};do python 5_association.py $i 2 0;done\nfor i in {91..101};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\n\nfor i in {00..101};do python 5_association.py $i 1 0,2;done\nfor i in {00..101};do py

In [2]:
plink_KCHIP_HLA_AA_SNP_1000G=PyPlink(plink_KCHIP_HLA_AA_SNP_1000G_path)
plink_KCHIP_HLA_AA_SNP_1000G_fam=plink_KCHIP_HLA_AA_SNP_1000G.get_fam().astype({'fid':str,'iid':str}).rename(columns={'fid':'FID','iid':'IID'})
plink_KCHIP_HLA_AA_SNP_1000G_bim=plink_KCHIP_HLA_AA_SNP_1000G.get_bim()

In [3]:
phenotypes=pd.read_csv(pheno_all_file_path,sep='\t')
phenotypes=phenotypes.set_index('ID').loc[plink_KCHIP_HLA_AA_SNP_1000G_fam['IID']]

binary_continuous_traits=sorted(phenotypes.columns[~phenotypes.columns.str.contains('x_ray')])    

In [4]:
gene_bed_path='data/mart_export_grch37.txt'
gene_bed=pd.read_csv(gene_bed_path,sep='\t')
gene_bed=gene_bed.drop(columns='Exon stable ID')
gene_bed=gene_bed[(gene_bed['Gene start (bp)']>=plink_KCHIP_HLA_AA_SNP_1000G_bim.pos.min())&(gene_bed['Gene end (bp)']<=plink_KCHIP_HLA_AA_SNP_1000G_bim.pos.max())]
gene_bed=gene_bed[(gene_bed['Transcript type']=='protein_coding')]
gene_bed=gene_bed[~gene_bed.duplicated(['Gene name','Gene start (bp)','Gene end (bp)'])]
print(gene_bed.shape)
gene_bed=gene_bed[~gene_bed.duplicated(['Gene name'])]
print(gene_bed.shape)

gene_assign=plink_KCHIP_HLA_AA_SNP_1000G_bim[['pos']]

for idx,row in gene_bed.iterrows():
    gene_assign[row['Gene name']]=0
    
for idx,row in gene_bed.iterrows():    
    gene_assign[row['Gene name']][(gene_assign['pos']>=row['Gene start (bp)'])&(gene_assign['pos']<=row['Gene end (bp)'])]=1

#gene_assign.columns=gene_assign.columns.str.replace('HLA-','HLA_')        
    
HLA_names=np.unique([i[0].split('_')[1] for i in plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_')].index.str.split('*')])

for HLA_name in HLA_names:
    gene_select=gene_assign[gene_assign.index.str.contains('HLA_'+HLA_name)|gene_assign.index.str.contains('SNPS_'+HLA_name)|gene_assign.index.str.contains('AA_'+HLA_name)]#print(gene_select.sort_values('pos').iloc[0],gene_select.sort_values('pos').iloc[-1])
    HLA_name='HLA-{}'.format(HLA_name)
    gene_assign[HLA_name][(gene_assign['pos']>=gene_select['pos'].min())&(gene_assign['pos']<=gene_select['pos'].max())]=1 

(209, 30)
(207, 30)


/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [42]:
gene_bed['Gene type'].unique(),gene_bed['Transcript type'].unique()

(array(['protein_coding'], dtype=object),
 array(['protein_coding'], dtype=object))

In [43]:
gene_bed

Gene stable ID Gene stable ID version Transcript stable ID  \
1367   ENSG00000197279      ENSG00000197279.3      ENST00000377325   
1380   ENSG00000196812      ENSG00000196812.4      ENST00000340487   
1391   ENSG00000198315      ENSG00000198315.6      ENST00000457389   
1430   ENSG00000137185      ENSG00000137185.7      ENST00000531941   
1458   ENSG00000187626      ENSG00000187626.7      ENST00000377294   
1468   ENSG00000189134      ENSG00000189134.3      ENST00000343684   
1481   ENSG00000137338      ENSG00000137338.4      ENST00000259883   
1498   ENSG00000235109      ENSG00000235109.3      ENST00000439158   
1566   ENSG00000189298      ENSG00000189298.9      ENST00000341464   
1584   ENSG00000158691     ENSG00000158691.10      ENST00000361028   
1600   ENSG00000187987      ENSG00000187987.5      ENST00000289788   
1610   ENSG00000198704      ENSG00000198704.5      ENST00000361902   
1624   ENSG00000224586      ENSG00000224586.2      ENST00000412168   
1643   ENSG00000232040      ENSG00000232040.2      ENST00000452236   
1678   ENSG00000204713      ENSG00000204713.6      ENST00000414543   
1714   ENSG00000204709      ENSG00000204709.4      ENST00000377186   
1723   ENSG00000197935      ENSG00000197935.6      ENST00000377179   
1737   ENSG00000204704      ENSG00000204704.2      ENST00000377175   
1740   ENSG00000204703      ENSG00000204703.3      ENST00000377173   
1741   ENSG00000204702      ENSG00000204702.4      ENST00000377171   
1742   ENSG00000204701      ENSG00000204701.1      ENST00000377169   
1746   ENSG00000204700      ENSG00000204700.3      ENST00000377167   
1759   ENSG00000204695      ENSG00000204695.2      ENST00000377160   
1761   ENSG00000243729      ENSG00000243729.3      ENST00000377154   
1766   ENSG00000112462      ENSG00000112462.8      ENST00000396806   
1767   ENSG00000168787      ENSG00000168787.4      ENST00000383555   
1769   ENSG00000204694      ENSG00000204694.6      ENST00000377148   
1777   ENSG00000206474      ENSG00000206474.6      ENST00000444197   
1778   ENSG00000204688      ENSG00000204688.5      ENST00000377136   
1803   ENSG00000204687      ENSG00000204687.3      ENST00000377127   
...                ...                    ...                  ...   
10782  ENSG00000204220      ENSG00000204220.5      ENST00000395131   
10859  ENSG00000237441      ENSG00000237441.5      ENST00000497454   
10926  ENSG00000231925      ENSG00000231925.7      ENST00000434618   
10979  ENSG00000236104      ENSG00000236104.2      ENST00000431845   
10985  ENSG00000204209      ENSG00000204209.6      ENST00000266000   
11038  ENSG00000237649      ENSG00000237649.3      ENST00000428849   
11064  ENSG00000112511     ENSG00000112511.13      ENST00000427004   
11159  ENSG00000112514     ENSG00000112514.11      ENST00000374500   
11234  ENSG00000197283      ENSG00000197283.8      ENST00000418600   
11335  ENSG00000213588      ENSG00000213588.4      ENST00000395064   
11338  ENSG00000030110      ENSG00000030110.8      ENST00000374467   
11358  ENSG00000204188      ENSG00000204188.3      ENST00000374458   
11366  ENSG00000096433      ENSG00000096433.6      ENST00000374316   
11484  ENSG00000137288      ENSG00000137288.5      ENST00000374231   
11496  ENSG00000269490      ENSG00000269490.1      ENST00000594414   
11498  ENSG00000161896      ENSG00000161896.6      ENST00000293756   
11511  ENSG00000161904      ENSG00000161904.7      ENST00000506578   
11582  ENSG00000096395      ENSG00000096395.6      ENST00000430124   
11619  ENSG00000124493      ENSG00000124493.9      ENST00000538487   
11736  ENSG00000137309     ENSG00000137309.15      ENST00000347617   
11773  ENSG00000186577      ENSG00000186577.7      ENST00000476320   
11812  ENSG00000272325      ENSG00000272325.1      ENST00000607016   
11817  ENSG00000270800      ENSG00000270800.1      ENST00000605528   
11826  ENSG00000124614      ENSG00000124614.9      ENST00000344700   
11859  ENSG00000124507      ENSG00000124507.6      ENST00000244458   
11901  ENSG0000012

In [41]:
gene_bed[gene_bed['Gene name'].str.contains('HLA')]

Gene stable ID Gene stable ID version Transcript stable ID  \
2188  ENSG00000204642      ENSG00000204642.9      ENST00000376861   
2317  ENSG00000204632      ENSG00000204632.7      ENST00000376828   
2393  ENSG00000206503      ENSG00000206503.7      ENST00000396634   
2953  ENSG00000204592      ENSG00000204592.5      ENST00000376630   
4716  ENSG00000204525     ENSG00000204525.10      ENST00000376228   
4764  ENSG00000234745      ENSG00000234745.5      ENST00000412585   
9245  ENSG00000204287      ENSG00000204287.9      ENST00000395388   
9257  ENSG00000198502      ENSG00000198502.5      ENST00000374975   
9284  ENSG00000196126      ENSG00000196126.6      ENST00000360004   
9290  ENSG00000196735      ENSG00000196735.7      ENST00000422863   
9324  ENSG00000179344     ENSG00000179344.12      ENST00000399082   
9367  ENSG00000237541      ENSG00000237541.3      ENST00000374940   
9373  ENSG00000232629      ENSG00000232629.4      ENST00000411527   
9399  ENSG00000241106      ENSG00000241106.2      ENST00000438763   
9564  ENSG00000242574      ENSG00000242574.4      ENST00000438510   
9589  ENSG00000204257     ENSG00000204257.10      ENST00000395305   
9781  ENSG00000204252      ENSG00000204252.8      ENST00000229829   
9803  ENSG00000231389      ENSG00000231389.3      ENST00000419277   
9830  ENSG00000223865      ENSG00000223865.6      ENST00000418931   

     Transcript stable ID version  Gene start (bp)  Gene end (bp)  \
2188            ENST00000376861.1         29690552       29706305   
2317            ENST00000376828.2         29794744       29798902   
2393            ENST00000396634.1         29909037       29913661   
2953            ENST00000376630.4         30457244       30461982   
4716            ENST00000376228.5         31236526       31239907   
4764            ENST00000412585.2         31321649       31324965   
9245            ENST00000395388.2         32407619       32412823   
9257            ENST00000374975.3         32485120       32498064   
9284            ENST00000360004.5         32546546       32557625   
9290            ENST00000422863.1         32595956       32614839   
9324            ENST00000399082.3         32627244       32636160   
9367            ENST00000374940.3         32709119       32714992   
9373            ENST00000411527.1         32723875       32731311   
9399            ENST00000438763.2         32780540       32784825   
9564            ENST00000438510.1         32902406       32908847   
9589            ENST00000395305.3         32916390       32936871   
9781            ENST00000229829.5         32971955       32977389   
9803            ENST00000419277.1         33032346       33048552   
9830            ENST00000418931.2         33043703       33054978   

      Transcript start (bp)  Transcript end (bp) Protein stable ID  \
2188               29690552             29694303   ENSP00000366057   
2317               29794744             29798902   ENSP00000366024   
2393               29909037             29913661   ENSP00000379873   
2953               30457244             30461982   ENSP00000365817   
4716               31236526             31239863   ENSP00000365402   
4764               31321649             31324964   ENSP00000399168   
9245               32407619             32412823   ENSP00000378786   
9257               32485120             32498064   ENSP00000364114   
9284               32546546             32557625   ENSP00000353099   
9290               32595956             32605308   ENSP00000405797   
9324               32627244             32634429   ENSP00000382032   
9367               32709119             32714992   ENSP00000364076   
9373               32723875             32731267   ENSP00000390431   
9399               32780540             32784825   ENSP00000390020   
9564               32902406             32905216   ENSP00000390848   
9589               32916390             32920838   ENSP00000378716   
9781               32971955             32977389   ENSP

In [39]:
gene_assign.loc['rs2517754']

KeyError: 'rs2517754'

In [38]:
gene_assign

pos  ZNF165  ZSCAN16  ZKSCAN8  ZSCAN9  \
snp                                                                     
6:28000361_T/C             28000361       0        0        0       0   
6:28000610_C/A             28000610       0        0        0       0   
6:28001003_A/C             28001003       0        0        0       0   
6:28001033_T/C             28001033       0        0        0       0   
6:28001610_T/C             28001610       0        0        0       0   
6:28002045_G/A             28002045       0        0        0       0   
6:28002139_G/A             28002139       0        0        0       0   
6:28002388_G/A             28002388       0        0        0       0   
6:28002818_C/T             28002818       0        0        0       0   
6:28002853_T/C             28002853       0        0        0       0   
6:28002963_A/G             28002963       0        0        0       0   
6:28003195_CTG/C           28003195       0        0        0       0   
6:28003205_G/A             28003205       0        0        0       0   
6:28003222_C/T             28003222       0        0        0       0   
6:28003275_T/G             28003275       0        0        0       0   
6:28004059_T/C             28004059       0        0        0       0   
6:28004078_A/G             28004078       0        0        0       0   
6:28004133_G/T             28004133       0        0        0       0   
6:28004134_G/A             28004134       0        0        0       0   
6:28004381_A/G             28004381       0        0        0       0   
6:28004835_A/G             28004835       0        0        0       0   
6:28005112_A/G             28005112       0        0        0       0   
6:28006042_A/G             28006042       0        0        0       0   
6:28006650_A/G             28006650       0        0        0       0   
6:28007357_A/C             28007357       0        0        0       0   
6:28007406_AG/A            28007406       0        0        0       0   
6:28007419_C/T             28007419       0        0        0       0   
6:28007595_CTT/C           28007595       0        0        0       0   
6:28007911_A/C             28007911       0        0        0       0   
6:28007930_C/A             28007930       0        0        0       0   
...                             ...     ...      ...      ...     ...   
6:34982237_TC/T            34982237       0        0        0       0   
6:34982599_C/T             34982599       0        0        0       0   
6:34983275_C/CA            34983275       0        0        0       0   
6:34984037_A/C             34984037       0        0        0       0   
6:34984556_A/AG            34984556       0        0        0       0   
6:34984558_A/G             34984558       0        0        0       0   
6:34984712_CTT/C           34984712       0        0        0       0   
6:34985133_TTCTC/T         34985133       0        0        0       0   
6:34985563_G/A             34985563       0        0        0       0   
6:34985985_G/A             34985985       0        0        0       0   
6:34986340_CA/C            34986340       0        0        0       0   
6:34986591_G/C             34986591       0        0        0       0   
6:34986717_A/G             34986717       0        0        0       0   
6:34988391_C/A             34988391       0        0        0       0   
6:34988951_A/C             34988951       0        0        0       0   
6:34989278_C/T             34989278       0        0        0       0   
6:34989598_A/G             34989598       0        0        0       0   
6:34989761_T/TA            34989761       0        0        0       0   
6:34990203_G/C             34990203       0        0        0       0   
6:34990968_C/T             34990968       0        0        0       0   
6:34993233_T/TCCCTTGCTGCC  34993233       0        0        0       0   
6:34994293_TTGA/T          34994293       0        0        0       0   
6:34994686_A/G   

In [5]:
#gene_assign.iloc[:,1:].sum(axis=1).sort_values()

In [6]:
"""
gene_bed_path='data/known_genes_chr6.hg19.txt'
gene_bed=pd.read_csv(gene_bed_path,sep='\t')
gene_bed=gene_bed[(gene_bed['txStart']>=plink_KCHIP_HLA_AA_SNP_1000G_bim.pos.min())&(gene_bed['txEnd']<=plink_KCHIP_HLA_AA_SNP_1000G_bim.pos.max())]
gene_bed=gene_bed[~gene_bed.duplicated(['name2'])]

gene_assign=plink_KCHIP_HLA_AA_SNP_1000G_bim[['pos']]

for idx,row in gene_bed.iterrows():
    gene_assign[row['name2']]=0
    
for idx,row in gene_bed.iterrows():    
    gene_assign[row['name2']][(gene_assign['pos']>=row['txStart'])&(gene_assign['pos']<=row['txEnd'])]=1
gene_assign=gene_assign.drop(columns='HLA-A29.1')

gene_assign.columns=gene_assign.columns.str.replace('HLA-','HLA_')        
    
HLA_names=np.unique([i[0].split('_')[1] for i in plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_')].index.str.split('*')])

for HLA_name in HLA_names:
    gene_select=gene_assign[gene_assign.index.str.contains('HLA_'+HLA_name)|gene_assign.index.str.contains('SNPS_'+HLA_name)|gene_assign.index.str.contains('AA_'+HLA_name)]#print(gene_select.sort_values('pos').iloc[0],gene_select.sort_values('pos').iloc[-1])
    HLA_name='HLA_{}'.format(HLA_name)
    gene_assign[HLA_name][(gene_assign['pos']>=gene_select['pos'].min())&(gene_assign['pos']<=gene_select['pos'].max())]=1 
"""    

"\ngene_bed_path='data/known_genes_chr6.hg19.txt'\ngene_bed=pd.read_csv(gene_bed_path,sep='\t')\ngene_bed=gene_bed[(gene_bed['txStart']>=plink_KCHIP_HLA_AA_SNP_1000G_bim.pos.min())&(gene_bed['txEnd']<=plink_KCHIP_HLA_AA_SNP_1000G_bim.pos.max())]\ngene_bed=gene_bed[~gene_bed.duplicated(['name2'])]\n\ngene_assign=plink_KCHIP_HLA_AA_SNP_1000G_bim[['pos']]\n\nfor idx,row in gene_bed.iterrows():\n    gene_assign[row['name2']]=0\n    \nfor idx,row in gene_bed.iterrows():    \n    gene_assign[row['name2']][(gene_assign['pos']>=row['txStart'])&(gene_assign['pos']<=row['txEnd'])]=1\ngene_assign=gene_assign.drop(columns='HLA-A29.1')\n\ngene_assign.columns=gene_assign.columns.str.replace('HLA-','HLA_')        \n    \nHLA_names=np.unique([i[0].split('_')[1] for i in plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_')].index.str.split('*')])\n\nfor HLA_name in HLA_names:\n    gene_select=gene_assign[gene_assign.index.str.contains('HLA_'+HLA_name)|gene_assign

In [7]:
#gene_assign.columns[gene_assign.columns.str.contains('HLA')]

In [ ]:
gene_assign_reduce=gene_assign.iloc[:,:1]

gene_assign_reduce['gene_1']=gene_assign.iloc[:,1:].apply(lambda x: x[x!=0].index[1-1] if (x!=0).sum()>=(1) else np.nan,axis=1)
gene_assign_reduce['gene_2']=gene_assign.iloc[:,1:].apply(lambda x: x[x!=0].index[2-1] if (x!=0).sum()>=(2) else np.nan,axis=1)
gene_assign_reduce['gene_3']=gene_assign.iloc[:,1:].apply(lambda x: x[x!=0].index[3-1] if (x!=0).sum()>=(3) else np.nan,axis=1)
gene_assign_reduce['gene_4']=gene_assign.iloc[:,1:].apply(lambda x: x[x!=0].index[4-1] if (x!=0).sum()>=(4) else np.nan,axis=1)

In [10]:
#gene_assign_reduce.to_csv('data/bim_to_gene_assign.tsv',sep='\t')

In [7]:
#gene_assign_reduce
gene_assign_reduce=pd.read_csv('data/bim_to_gene_assign.tsv',sep='\t',index_col=0)

In [23]:
result_all=pd.read_csv('data/result_concat_1_top_all.tsv',sep='\t')
result_all['P_significant']=(result_all['P']<5e-8).astype(int)

In [24]:
result_all.head()

Unnamed: 0  Unnamed: 0.1 phenotype_type          phenotype_name  \
0       20816        144613     continuous  FEV_over_FVC_predicted   
1       68685        475104     continuous           FEV_predicted   
2       21318        169288     continuous           FEV_predicted   
3       68741        475496     continuous           FVC_predicted   
4       21307        169200     continuous           FVC_predicted   

  samples(case/control)  step             condition           marker_name  \
0                  7440     1                   NaN        6:30985177_G/A   
1                  7440     1                   NaN        6:34172055_C/A   
2                  7440     2        6:34172055_C/A        6:31018788_T/G   
3                  7441     1                   NaN  6:34205465_G/GGAGCCC   
4                  7441     2  6:34205465_G/GGAGCCC           AX-11396301   

        marker_name_pub               note  ... A1_freq_control  multi_allele  \
0            rs12204947  unphased bialleic  ...             NaN           NaN   
1             rs1187115  unphased bialleic  ...             NaN           NaN   
2             rs2517531  unphased bialleic  ...             NaN           NaN   
3  6:34205465_G/GGAGCCC  unphased bialleic  ...             NaN           NaN   
4             rs2517534  unphased bialleic  ...             NaN           NaN   

   nobs        Z      coef       std chisq  df             P  P_significant  
0  7440 -5.18377 -0.060131  0.011600   NaN NaN  2.231720e-07              0  
1  7440  5.83644  0.026954  0.004618   NaN NaN  5.556790e-09              1  
2  7440 -4.35216 -0.015100  0.003470   NaN NaN  1.366070e-05              0  
3  7441  5.57356  0.036813  0.006605   NaN NaN  2.583340e-08              1  
4  7433  4.63010  0.022921  0.004950   NaN NaN  3.717070e-06              0  

[5 rows x 25 columns]

In [25]:
len(result_all['marker_name']),len(result_all[result_all['P_significant']==1]['marker_name'].unique()),len(result_all[result_all['P_significant']==1]['marker_name_pub'].unique())

(168, 67, 67)

In [26]:
#result_all[result_all['P_significant']==1]['marker_name_pub']

In [27]:
r2_df=pd.read_csv('data/r2.tsv',sep='\t',index_col=0)

In [28]:
gene_assign.columns[gene_assign.columns.str.contains('HLA')]

Index(['HLA-F', 'HLA-G', 'HLA-A', 'HLA-E', 'HLA-C', 'HLA-B', 'HLA-DRA',
       'HLA-DRB5', 'HLA-DRB1', 'HLA-DQA1', 'HLA-DQB1', 'HLA-DQA2', 'HLA-DQB2',
       'HLA-DOB', 'HLA-DMB', 'HLA-DMA', 'HLA-DOA', 'HLA-DPA1', 'HLA-DPB1'],
      dtype='object')

In [29]:
'HLA-A', 'HLA-C', 'HLA-B'

'HLA-DRA','HLA-DRB1', 'HLA-DQA1', 'HLA-DQB1','HLA-DPA1', 'HLA-DPB1'

('HLA-DRA', 'HLA-DRB1', 'HLA-DQA1', 'HLA-DQB1', 'HLA-DPA1', 'HLA-DPB1')

In [30]:
"""
def marker_name_to_gene(marker_name):
    if marker_name[:3]=='AA_':
        ret='HLA_'+marker_name.split('_')[1]
    elif marker_name[:5]=='SNPS_':
        ret='HLA_'+marker_name.split('_')[1]
        #conditional_list.append('HLA_'+marker_name.split('_')[1])
    elif marker_name[:4]=='HLA_':
        ret='HLA_'+marker_name.split('_')[1].split('*')[0]
    elif marker_name[:9]=='INS_SNPS_':
        ret='HLA_'+marker_name.split('_')[2]
    elif np.any(~gene_assign_reduce.loc[marker_name].iloc[1:].isnull()):
        ret=gene_assign_reduce.loc[marker_name].iloc[1:]
        ret=','.join(ret[~ret.isnull()])
    else:
        r2=r2_df[marker_name].sort_values(ascending=False)
        gene_assign_r2_moderate_HLA=gene_assign.loc[r2[r2>0.5].index][gene_assign.columns[gene_assign.columns.str.contains('HLA')]]
        gene_assign_r2_moderate_HLA=gene_assign_r2_moderate_HLA[gene_assign_r2_moderate_HLA.sum(axis=1)!=0]
        #print(gene_assign_r2_moderate_HLA.sum(axis=1)!=0)
        
        gene_assign_r2_strong=gene_assign.loc[r2[r2>0.8].index].iloc[:,1:]
        gene_assign_r2_strong=gene_assign_r2_strong[gene_assign_r2_strong.sum(axis=1)!=0]
        #gene_assign_r2_strong_sum=gene_assign_r2_strong.sum()
        #
        if len(gene_assign_r2_moderate_HLA)!=0:
            ret=gene_assign_r2_moderate_HLA.iloc[0].sort_values(ascending=False).index[0]
        elif len(gene_assign_r2_strong)!=0:
            ret=gene_assign_r2_strong.iloc[0].sort_values(ascending=False).index[0]#[gene_assign_r2_strong.iloc[0]==1]
        else:
            #print(marker_name)
            gene_assign_temp=gene_assign.copy()
            gene_assign_temp['dist']=(gene_assign_temp['pos']-gene_assign_temp['pos'][marker_name]).abs()
            #print(marker_name,gene_assign_temp.sort_values('dist'))
            
            gene_assign_temp_clip=gene_assign_temp.sort_values('dist').iloc[:,1:-1]
            
            for row_idx,row in gene_assign_temp_clip.iterrows():
                if np.any(row!=0):
                    #print(row)
                    ret=','.join(row[row!=0].index)
                    break
    return ret
"""    

"\ndef marker_name_to_gene(marker_name):\n    if marker_name[:3]=='AA_':\n        ret='HLA_'+marker_name.split('_')[1]\n    elif marker_name[:5]=='SNPS_':\n        ret='HLA_'+marker_name.split('_')[1]\n        #conditional_list.append('HLA_'+marker_name.split('_')[1])\n    elif marker_name[:4]=='HLA_':\n        ret='HLA_'+marker_name.split('_')[1].split('*')[0]\n    elif marker_name[:9]=='INS_SNPS_':\n        ret='HLA_'+marker_name.split('_')[2]\n    elif np.any(~gene_assign_reduce.loc[marker_name].iloc[1:].isnull()):\n        ret=gene_assign_reduce.loc[marker_name].iloc[1:]\n        ret=','.join(ret[~ret.isnull()])\n    else:\n        r2=r2_df[marker_name].sort_values(ascending=False)\n        gene_assign_r2_moderate_HLA=gene_assign.loc[r2[r2>0.5].index][gene_assign.columns[gene_assign.columns.str.contains('HLA')]]\n        gene_assign_r2_moderate_HLA=gene_assign_r2_moderate_HLA[gene_assign_r2_moderate_HLA.sum(axis=1)!=0]\n        #print(gene_assign_r2_moderate_HLA.sum(axis=1)!=0)\n  

In [31]:
def marker_name_to_genehow(marker_name):
    if marker_name[:3]=='AA_':
        ret=('HLA-'+marker_name.split('_')[1],'on_gene')
    elif marker_name[:5]=='SNPS_':
        ret=('HLA-'+marker_name.split('_')[1],'on_gene')
        #conditional_list.append('HLA_'+marker_name.split('_')[1])
    elif marker_name[:4]=='HLA_':
        ret=('HLA-'+marker_name.split('_')[1].split('*')[0],'on_gene')
    elif marker_name[:9]=='INS_SNPS_':
        ret=('HLA-'+marker_name.split('_')[2],'on_gene')
    elif np.any(~gene_assign_reduce.loc[marker_name].iloc[1:].isnull()):
        ret=gene_assign_reduce.loc[marker_name].iloc[1:]
        ret=(','.join(ret[~ret.isnull()]),'on_gene')
    else:
        r2=r2_df[marker_name].sort_values(ascending=False)
        gene_assign_r2_moderate_HLA=gene_assign.loc[r2[r2>0.5].index][gene_assign.columns[gene_assign.columns.str.contains('HLA')].tolist()+['MICA','MICB','TAP1','TAP2']]
        gene_assign_r2_moderate_HLA=gene_assign_r2_moderate_HLA[gene_assign_r2_moderate_HLA.sum(axis=1)!=0]
        #print(gene_assign_r2_moderate_HLA.sum(axis=1)!=0)
        
        gene_assign_r2_strong=gene_assign.loc[r2[r2>0.8].index].iloc[:,1:]
        gene_assign_r2_strong=gene_assign_r2_strong[gene_assign_r2_strong.sum(axis=1)!=0]
        #gene_assign_r2_strong_sum=gene_assign_r2_strong.sum()
        #
        if len(gene_assign_r2_moderate_HLA)!=0:
            ret=gene_assign_r2_moderate_HLA.iloc[0].sort_values(ascending=False).index[0]
            ret=(ret,'moderate_LD')
            print(marker_name)
            print(gene_assign_r2_moderate_HLA.iloc[:10])
        elif len(gene_assign_r2_strong)!=0:
            ret=gene_assign_r2_strong.iloc[0].sort_values(ascending=False).index[0]#[gene_assign_r2_strong.iloc[0]==1]
            ret=(ret,'strong_LD')
        else:
            #print(marker_name)
            gene_assign_temp=gene_assign.copy()
            gene_assign_temp['dist']=(gene_assign_temp['pos']-gene_assign_temp['pos'][marker_name]).abs()
            #print(marker_name,gene_assign_temp.sort_values('dist'))
            
            gene_assign_temp_clip=gene_assign_temp.sort_values('dist').iloc[:,1:-1]
            
            for row_idx,row in gene_assign_temp_clip.iterrows():
                if np.any(row!=0):
                    #print(row)
                    ret=','.join(row[row!=0].index)
                    ret=(ret,'nearest')
                    break
    return ret

In [32]:
result_all[result_all['phenotype_name']=='hdl_cholesterol']
result_all[result_all['phenotype_name']=='bun']

Unnamed: 0  Unnamed: 0.1 phenotype_type phenotype_name  \
33       18966        169281     continuous            bun   
34       13105        323700     continuous            bun   

   samples(case/control)  step             condition     marker_name  \
33                124563     1                   NaN  6:30737591_G/C   
34                124563     2  6:30737591_G/C,HLA_C  6:29922341_G/C   

   marker_name_pub               note  ... A1_freq_control  multi_allele  \
33       rs3095337  unphased bialleic  ...             NaN           NaN   
34      rs28780070  unphased bialleic  ...             NaN           NaN   

      nobs        Z      coef       std chisq  df             P  P_significant  
33  124563 -6.93374 -0.186481  0.026895   NaN NaN  4.118390e-12              1  
34  124563  4.52102  0.071743  0.015869   NaN NaN  6.159850e-06              0  

[2 rows x 25 columns]

In [33]:
result_all['gene_genehow']=np.nan
result_all['gene']=np.nan
result_all['genehow']=np.nan

result_all['gene_genehow'][result_all['P_significant']==1]=\
result_all[result_all['P_significant']==1].apply(lambda x: marker_name_to_genehow(x['marker_name']),axis=1)

result_all['gene'][result_all['P_significant']==1]=result_all[result_all['P_significant']==1].apply(lambda x: x['gene_genehow'][0],axis=1)
result_all['genehow'][result_all['P_significant']==1]=result_all[result_all['P_significant']==1].apply(lambda x: x['gene_genehow'][1],axis=1)

AX-41950637
                             HLA-F  HLA-G  HLA-A  HLA-E  HLA-C  HLA-B  \
snp                                                                     
SNPS_C_544_31239305_intron2      0      0      0      0      1      0   
SNPS_C_514_31239335_intron2      0      0      0      0      1      0   

                             HLA-DRA  HLA-DRB5  HLA-DRB1  HLA-DQA1  ...  \
snp                                                                 ...   
SNPS_C_544_31239305_intron2        0         0         0         0  ...   
SNPS_C_514_31239335_intron2        0         0         0         0  ...   

                             HLA-DOB  HLA-DMB  HLA-DMA  HLA-DOA  HLA-DPA1  \
snp                                                                         
SNPS_C_544_31239305_intron2        0        0        0        0         0   
SNPS_C_514_31239335_intron2        0        0        0        0         0   

                             HLA-DPB1  MICA  MICB  TAP1  TAP2  
snp                  

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [34]:
result_all[result_all['genehow']=='moderate_LD'].head()

Unnamed: 0  Unnamed: 0.1 phenotype_type  phenotype_name  \
7        23270        208017     continuous         albumin   
13       30669        272754     continuous             alt   
18       35667        317736     continuous             ast   
19       63263       1498800     continuous             ast   
25       11573         92584         binary  blood_in_urine   

   samples(case/control)  step                condition     marker_name  \
7                 125069     1                      NaN     AX-41950637   
13                120788     1                      NaN  6:31351530_T/C   
18                121162     1                      NaN  6:32202630_G/T   
19                121162     2  6:32202630_G/T,HLA_DQB1  6:33076268_T/C   
25            1817/14938     1                      NaN  6:29896680_A/G   

   marker_name_pub               note  ...        Z      coef       std chisq  \
7        rs7762933  unphased bialleic  ... -8.55617 -0.008719  0.001019   NaN   
13       rs2905715  unphased bialleic  ...  6.52296  0.326987  0.050129   NaN   
18     rs117654833  unphased bialleic  ...  7.60265  0.443845  0.058380   NaN   
19        rs742870  unphased bialleic  ... -5.81671 -0.191552  0.032931   NaN   
25       rs2517754  unphased bialleic  ... -5.53651 -0.201425  0.036381   NaN   

    df             P P_significant             gene_genehow      gene  \
7  NaN  1.179740e-17             1     (HLA-C, moderate_LD)     HLA-C   
13 NaN  6.920490e-11             1      (MICA, moderate_LD)      MICA   
18 NaN  2.921930e-14             1  (HLA-DRB1, moderate_LD)  HLA-DRB1   
19 NaN  6.016610e-09             1  (HLA-DPB1, moderate_LD)  HLA-DPB1   
25 NaN  3.085650e-08             1     (HLA-A, moderate_LD)     HLA-A   

        genehow  
7   moderate_LD  
13  moderate_LD  
18  moderate_LD  
19  moderate_LD  
25  moderate_LD  

[5 rows x 28 columns]

In [35]:
result_all

Unnamed: 0  Unnamed: 0.1 phenotype_type  \
0         20816        144613     continuous   
1         68685        475104     continuous   
2         21318        169288     continuous   
3         68741        475496     continuous   
4         21307        169200     continuous   
5         29549        204407     continuous   
6         56431        445472         binary   
7         23270        208017     continuous   
8         68761        679480     continuous   
9         22997        251240     continuous   
10        17705        210576     continuous   
11        54209        481860         binary   
12        12809        290996         binary   
13        30669        272754     continuous   
14        68693        678800     continuous   
15        51373           419     continuous   
16        29461           198         binary   
17        36489        325134         binary   
18        35667        317736     continuous   
19        63263       1498800     continuous   
20        68693       2307920     continuous   
21        38500        343233         binary   
22          671          4697         binary   
23        37231        258076         binary   
24        64351        508304         binary   
25        11573         92584         binary   
26        38146        340047         binary   
27        51369           417     continuous   
28         4332         95304     continuous   
29        68875       1565426     continuous   
..          ...           ...            ...   
138       41739           794     continuous   
139       68714        611109         binary   
140       35422        350590         binary   
141       34182        304371         binary   
142       51056        454437         binary   
143       32232        701118         binary   
144       33689        766498         binary   
145       55587       1316160         binary   
146       30563        755000         binary   
147       39995        356688     continuous   
148       54447           494     continuous   
149       61399       1394996     continuous   
150       29520           167         binary   
151       39841        355302     continuous   
152       68714       1561723     continuous   
153       34658        823080     continuous   
154       69788        482825     continuous   
155        3634         25438         binary   
156       11836        106524         binary   
157       68715        611118     continuous   
158       25152        224775     continuous   
159       42966       1021944     continuous   
160       32222       1401796     continuous   
161       68824       3060495     continuous   
162       11583        532818     continuous   
163       68741        611352     continuous   
164       47363        468200     continuous   
165        3225         77400     continuous   
166       20964        520175     continuous   
167       11868        308568     continuous   

                            phenotype_name samples(case/control)  step  \
0                   FEV_over_FVC_predicted                  7440     1   
1                            FEV_predicted                  7440     1   
2                            FEV_predicted                  7440     2   
3                            FVC_predicted                  7441     1   
4                            FVC_predicted                  7441     2   
5                            MVV_predicted                  7441     1   
6                      acute_liver_disease             669/74104     1   
7                                  albumin                125069     1   
8                                  albumin                125069     2   
9                                  albumin                125069     3   
10                                 albumin                125069     4   
11                        allergic_disease            9273/78950     1   
12                        allergic_disease            9273/78950  

In [45]:
result_all[result_all['P_significant']==1]

Unnamed: 0  Unnamed: 0.1 phenotype_type            phenotype_name  \
1         68677        475104     continuous             FEV_predicted   
3         68733        475496     continuous             FVC_predicted   
7         23267        208017     continuous                   albumin   
8         68753        679480     continuous                   albumin   
9         22994        251240     continuous                   albumin   
11        54203        481860         binary          allergic_disease   
13        30666        272754     continuous                       alt   
14        68685        678800     continuous                       alt   
18        35664        317736     continuous                       ast   
19        63255       1498800     continuous                       ast   
25        11573         92584         binary            blood_in_urine   
27        51364           417     continuous                       bmi   
28         4332         95304     continuous                       bmi   
33        18963        169281     continuous                       bun   
46        31181        277389     continuous                creatinine   
47        25121        598824     continuous                creatinine   
49        35371        315099     continuous  diastolic_blood_pressure   
58        31181        215747     continuous             grip_strength   
61        30860        274500     continuous                     hba1c   
63        38054        339246     continuous           hdl_cholesterol   
64        37991        376310     continuous           hdl_cholesterol   
65        70668        768493     continuous           hdl_cholesterol   
68        68738        611397     continuous                    height   
69         5776         57760     continuous                    height   
70        22772        248798     continuous                    height   
71        71028       1896021     continuous                    height   
72         4149        116172     continuous                    height   
74        35289        314361     continuous                hematocrit   
75        29374        291010     continuous                hematocrit   
76         7442        267912     continuous                hematocrit   
..          ...           ...            ...                       ...   
110       65631        583434     continuous                  platelet   
111       65627        648220     continuous                  platelet   
112       24319        265815     continuous                  platelet   
113       45013           660     continuous                  platelet   
114       68707       2172864     continuous                  platelet   
115       12341             4     continuous                  platelet   
116       29754       1322730     continuous                  platelet   
119       36851        291928     continuous          protein_in_blood   
122        6018         54162     continuous                     pulse   
125       29492           179     continuous                       rbc   
126       68735       2377550     continuous                       rbc   
127       35164       1252944     continuous                       rbc   
128       13551        495689     continuous                       rbc   
137       35371        315099     continuous   systolic_blood_pressure   
139       68706        611109         binary               t2_diabetes   
142       51051        454437         binary           thyroid_disease   
143       32229        701118         binary           thyroid_disease   
144       33685        766498         binary           thyroid_disease   
145       55581       1316160         binary           thyroid_disease   
148       54439           494     continuous         total_cholesterol   
151       39838        355302     continuous              triglyceride   
152       68706       1561723     continuous              triglyceride   
158      

In [36]:
result_all.to_csv('data/result_concat_2_top_gene_annot.tsv',sep='\t',index=None)

In [100]:
#result_all[result_all['phenotype_name']=='glucose_in_blood']

In [286]:
for i in result_all['gene'][result_all['step']!=1]:
    print(i)

HCG22
HCG22
HMGA1
HLA_E
HLA_A
HMGA1
HLA_DQA1
HLA_DPB1
HMGA1
HLA_A
TRIM27
C6orf1
HLA_A
NOTCH4
HLA_DRB1
HMGA1
NOTCH4
ITPR3
HLA_B
HMGA1
UHRF1BP1
HCG22
HLA_DPB1
HLA_C
SCAND3
TAF11
IER3
LSM2
HLA_B
LOC285830
FLJ43752
HCG9
HLA_DRB1
TRIM27
RPS10
HCG26
HMGA1
GRM4
GRM4
GRM4
ZSCAN16
HLA_B
BAK1
HLA_DRB1
HMGA1
HLA_A
HLA_DPB1
ZFP57
HMGA1
ATF6B
HLA_A
HLA_B
HLA_DRB1
HLA_B
HLA_C
HLA_DOA
HMGA1
SKIV2L
HLA_DRB1
HLA_B
HMGA1
HLA_A
HLA_DQA1
TRIM27
HCG22
HLA_A
